In [1]:
import pandas as pd
import regex as re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/ranggi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data MUA for Memory Based System Recommendation

In [2]:
class ContentBasedFiltering:
    def case_folding(self, text):
        if type(text) == str:
            text = text.lower()
        return text

    def punctuation_removal(self, text):
        if type(text) == str:
            text = re.sub(r'[^\w\s]', '', text)
        return text

    def tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens
        
    def stopwords_removal(self, text):
        stopword_factory = StopWordRemoverFactory()
        stopword_remover = stopword_factory.create_stop_word_remover()
        no_stopword = stopword_remover.remove(text)
        return no_stopword

    def preprocessing(self, text):
        text = self.case_folding(text)
        text = self.punctuation_removal(text)
        text = self.stopwords_removal(text)
        return text

In [3]:
data = pd.read_excel('./web_mua/static/Data MUA 2.xlsx')
data.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating
0,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,natural (putih),light,cool,Murah,-8.233613,113.66733,4
1,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,4
2,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,rachel 1 (kuning langsat),medium,neutral,Murah,-8.233613,113.66733,5
3,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,5
4,3,septia maharani,Rias Pengantin Azizah,-8.234863,113.680670,la tulipe cover foundation,natural (putih),light,cool,Sedang,-8.233613,113.66733,3


In [4]:
def cleaning_data(data):
    cleaned_data = []
    CBF = ContentBasedFiltering()
    for i in range(len(data)):
        cleaned_data.append(CBF.preprocessing(str(data[i])))
    return cleaned_data

In [5]:
#praproses data untuk nama
data['nama'] = cleaning_data(data['nama'])

In [6]:
data_for_model = data.copy()
data_for_user = data.copy()

#praproses data untuk deskripsi
data_for_model['desc'] = data['produk_makeup'] + ' ' + data['shade'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_model['desc'] = cleaning_data(data_for_model['desc'])
data_for_user['desc'] = data['produk_makeup'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_user['desc'] = cleaning_data(data_for_user['desc'])

In [7]:
# data_for_model = data_for_model.groupby('no').agg({'nama':'first', 'nama_MUA':'first', 'kategori_harga': 'first', 'latitude':'first', 'longtitude':'first', 'rating':'first','desc':' '.join}).reset_index()
data_for_model = data_for_model.sort_values(by=['nama'], ignore_index=True)

data_for_user = data_for_user.sort_values(by=['nama'], ignore_index=True)
data_for_user = data_for_user.drop(columns=['shade'])

In [8]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [9]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


In [10]:
data_for_model.shape

(179, 14)

In [11]:
from sklearn.preprocessing import LabelEncoder

harga_labels = LabelEncoder()
harga_labels = harga_labels.fit(data['kategori_harga'])
harga_labels.classes_ = np.array(['Murah', 'Sedang', 'Mahal'])
data_for_model['kategori_harga'] = harga_labels.transform(data_for_model['kategori_harga'])
data_for_user['kategori_harga'] = harga_labels.transform(data_for_user['kategori_harga'])

In [12]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [13]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


## User Input

In [14]:
# use this parameter to get some MUA recommendations
def data_preparation(data_for_model, data_for_user, index_user, user):
    du = data_for_user.copy()
    query_data = du.iloc[[index_user]].reset_index()
    query_data_todict = query_data.to_dict('index')[0]
    part_of_model = data_for_model.copy()
    part_of_model = part_of_model.loc[part_of_model.nama_MUA != query_data.nama_MUA[0]].reset_index()
    part_of_query = part_of_model.loc[part_of_model['nama'] == user].reset_index()
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return query_data_todict, part_of_query, part_of_model

# example of input data
find_user = data_for_user.index[data_for_user['nama'] == 'adera zia'].to_list()
txt_user = 'adera zia'
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user)
# part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user, first=False)

## Calculating Cosine Similarity for Content Based Model

In [190]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF

def tf_idf_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(md["desc"])
    nmf_model = NMF(10)
    doc_topic = nmf_model.fit_transform(X)
    model = doc_topic[:len(part_of_model)]
    query = doc_topic[len(part_of_model):]
    return model, query

In [191]:
from geopy.distance import great_circle
import math
def calculate_distance(query_data_todict, part_of_model):
        loc1 = (query_data_todict['latitude'], query_data_todict['longtitude'])
        distance_loc = []
        for i in range(len(part_of_model.index)):
            loc2 = (part_of_model['latitude'][i], part_of_model['longtitude'][i])
            tmp = great_circle(loc1, loc2).km
            value = math.ceil(tmp * 100) / 100
            distance_loc.append(value)
        return distance_loc

In [192]:
def get_numtext_content(query_data_todict, part_of_model):
    tfidf_model, tfidf_query = tf_idf_vector(part_of_model, query_data_todict)
    cos_sim = cosine_similarity(tfidf_query, tfidf_model)
    data_sim = cos_sim.reshape(-1,1)
    distance = calculate_distance(query_data_todict, part_of_model)
    return data_sim, distance

In [193]:
from sklearn.preprocessing import StandardScaler

def get_values(part_of_model):
    tmp_data = part_of_model[['kategori_harga','sim','distance']].to_numpy()
    sim_num = cosine_similarity(tmp_data)
    sim_num = np.array([np.mean(p) for p in sim_num])
    return sim_num

In [194]:
def value_of_mua(query_data_todict, part_of_model):
    sim, distance = get_numtext_content(query_data_todict, part_of_model)
    part_of_model["sim"] = sim
    part_of_model["distance"] = distance
    sf = []
    for i in range(len(part_of_model)):
        sf.append(part_of_model["sim"][i]/(part_of_model["distance"][i] + 1))
    part_of_model["sim_final"] = sf   
    sim_num = get_values(part_of_model)
    part_of_model["sim_num"] = sim_num
    part_of_model = part_of_model.sort_values(by=['nama_MUA'])
    nama_mua = part_of_model["nama_MUA"].unique()
    value_mua = np.empty(len(nama_mua), dtype=float)
    for i in range(len(nama_mua)):
        tmp_val = []
        for j in range(len(part_of_model.index)):
            if nama_mua[i] == part_of_model.nama_MUA[j]:
                tmp_val.append(part_of_model["sim_num"][j] + part_of_model["sim_final"][j])
        value_mua[i] = np.mean(tmp_val)
    part_of_model['total_mua'] = part_of_model["sim_num"] + part_of_model["sim_final"]
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return nama_mua, value_mua, part_of_model

## FIND USER

In [195]:
unique_user = data_for_model['nama'].unique()
unique_user
query_user = ['adera zia', 'anggia pradita a', 'anissa serina', 'calista devi aqilah', 'callista febriana erwanto', 'dwi rahma', 'maulina vira', 
              'vika aprilia sinta putri', 'yuricha mahargiyani wijaya', 'zainata hanum masruroh', 'zyanti laila']
selected_user = []
for i in query_user:
    find_user = data_for_model.index[data_for_model['nama'] == i].to_list()
    selected_user.append(find_user[0])
selected_user

[0, 10, 20, 33, 45, 73, 101, 142, 151, 161, 169]

## HYBRID FILTERING

In [196]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing as pre
import statistics 

In [197]:
#chosen user for testing
def hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user):
    actual_hyb = []
    table_hyb = []
    rating_hyb = []
    for index_user in range(len(selected_user)):
        query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        
        nmf_model = NMF(2)
        f = nmf_model.fit_transform([feature])
        m = nmf_model.fit_transform(val_model)
        cos_sim_user = cosine_similarity(f, m)

        weight_user = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_user)):
                if part_of_model["nama"][i] == nama_user[j]:
                    weight_user[i] = cos_sim_user[0][j]
        
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA']]
        final_res['user'] = weight_user
        final_res['content'] = weight_content
        
        mean_weight = np.zeros(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
        final_res['mean_'] = mean_weight
        higher_value = final_res.copy()
        higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
        higher_value['weight_user'] = u['mean']
        c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
        higher_value['weight_content'] = c['mean']
        higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
        list_nama = higher_value.nama[:10].tolist()
        pivot = part_of_model.copy()
        pivot['mean_'] = mean_weight
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_['weight']
        pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        if pivot_['weight'][i] != 0.0:
                            value_rating[j] = pivot_["mean"][i] * pivot_['weight'][i]
                        else:
                            value_rating[j] = weight_content[i]
                else:
                    continue
        
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break

        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        
        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_hyb.append(final_label)
        table_hyb.append(pivot_)
        rating_hyb.append(value_rating)

    #     mae = mean_absolute_error(value_rating, final_actual[index_user])
    #     mae_hy.append(mae)
        
    # mean_mae_hy = np.mean(mae_hy)
    return table_hyb, rating_hyb, actual_hyb, col_model

In [198]:
hybrid_based_fil, value_rating_hyb, actual_hyb, col_model = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [199]:
hybrid_based_fil[1]

,count,mean,weight,predicted
nama_MUA,,,,
Wedding Gallery Make Over,2,5.000000,0.896724,4.483619
Winda Wijaya MUA & Beauty Studio,3,5.000000,0.893928,4.469642
Sanggar Rias Cantik Tina,2,5.000000,0.892281,4.461405
Laura Make Up & Bridal,3,4.333333,0.961690,4.167323
Ulfa Beauty MUA,3,4.666667,0.891666,4.161108
Griya Putri Make Up,6,4.333333,0.927989,4.021284
Intan Rias Pengantin,2,4.000000,1.003485,4.013940
Eka Make Over,2,4.000000,0.988438,3.953752
Dinda Wijaya Make Up,4,4.000000,0.969126,3.876503


In [149]:
col_model

['Abha Beauty Make Up Artist',
 'Aira Salon',
 'Arachma Make Up',
 'Ardie Make Up & Salon',
 'Aulia Rias Pengantin',
 'Ayuk Rias Pengantin',
 'Ayunia Bridal dan Salon',
 'Bonita MUA Jember',
 'Dinda Wijaya Make Up',
 'Eka Make Over',
 'Ella Surya Make Up',
 'Gasebo Make Up Dekorasi',
 'Griya Citra Ayu',
 'Griya Putri Make Up',
 'Hiekmaa MUA',
 'Ifa Salon',
 'Intan Rias Pengantin',
 'Iqlim Rias Pengantin',
 'KL Make Up Private Studio',
 'Khaliza Make Up',
 'Larasnada Make Up',
 'Laura Make Up & Bridal',
 'Liyan Make Up Jember',
 'MUA Niara Puspita',
 'Nonarika Make Up & Hairdo',
 'Novi Make Up',
 'RR Make Up',
 'Rias Manten Mariyam',
 'Rias Ning Lida',
 'Rias Pengantin Ana',
 'Rias Pengantin Azizah',
 'Rias Pengantin Criepno Lestari',
 'Rias Pengantin Sapphire',
 'Rosa Indah Griya Rias Pengantin',
 'Salon Fatah',
 'Salon Indah Sari',
 'Salon Puput Jember',
 'Salon Sriwijaya',
 'Salon Sylvia',
 'Salon Vinell',
 'Sanggar Rias Busana Kartika Budaya',
 'Sanggar Rias Cantik Tina',
 'Sanggar 

## TESTING

In [92]:
def testing_user(data_for_model, data_for_user, index_usr):
    query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_usr], query_user[index_usr])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]
        
    weight_content = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_mua)):
            if part_of_model["nama_MUA"][i] == nama_mua[j]:
                weight_content[i] = value_mua[j]
        
    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user
    final_res['content'] = weight_content

    mean_weight = np.zeros(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
    final_res['mean_'] = mean_weight
    higher_value = final_res.copy()
    higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
    u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
    higher_value['weight_user'] = u['mean']
    c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
    higher_value['weight_content'] = c['mean']
    higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
    print(higher_value)
    list_nama = higher_value.nama[:10].tolist()
    pivot = part_of_model.copy()
    pivot['mean_'] = mean_weight
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    distance = pivot[['nama_MUA','distance']]
    produk = pivot[['nama_MUA', 'sim']]
    final_ = pivot[['nama_MUA', 'sim_final']]
    desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
    desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
    pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
    pivot_['score'] = pivot_["mean"] * pivot_['weight']
    pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
    pivot_['distance'] = distance.groupby("nama_MUA").distance.agg(['mean'])
    pivot_['sim'] = produk.groupby("nama_MUA").sim.agg(['mean'])
    pivot_['sim_final'] = final_.groupby("nama_MUA").sim_final.agg(['mean'])
    pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)
    return pivot_

In [93]:
#highest MAE
mae_eval[mae_eval.index(min(mae_eval))]

0.7345715161925662

In [94]:
#highest
idx_high = mae_eval.index(min(mae_eval))
data_for_model.iloc[[selected_user[idx_high]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
45,43,callista febriana erwanto,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,0,-8.157255,113.652074,4,la tulipe cover foundation suntan sawo matang ...


In [95]:
h_pivot = testing_user(data_for_model, data_for_user, idx_high)
h_pivot[:9]

                              nama      mean  weight_user  weight_content
0        callista febriana erwanto  1.105529     1.000000        0.996759
1                    eka febrianti  0.883173     0.545545        1.006900
2               rodhiyyatul hikmah  0.877757     0.545545        0.992929
3       yuricha mahargiyani wijaya  0.680082     0.268608        1.022355
4              calista devi aqilah  0.649471     0.240981        0.995443
5             donna rachel elkarim  0.394346     0.000000        1.084610
6                       dinda dira  0.393504     0.000000        1.071994
7                      dea wanda p  0.392221     0.000000        1.053229
8               trevina wigianiska  0.392187     0.000000        1.052738
9                  indah febrianti  0.391986     0.000000        1.049849
10                   alevia frista  0.391254     0.000000        1.039423
11          zainata hanum masruroh  0.391253     0.000000        1.039517
12          najwa anatasya pribadi  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim,sim_final
0,5,4.600000,Rias Manten Mariyam,1.020321,4.693477,[la tulipe cover foundation - rachel 1 (kuning...,1.91,0.274022,0.094166
1,5,4.600000,Eka Make Over,0.929164,4.274154,[la tulipe cover foundation - natural (putih) ...,54.12,0.338732,0.006145
2,7,4.571429,Salon Fatah,0.844098,3.858734,[la tulipe cover foundation - suntan (sawo mat...,33.67,0.362959,0.010469
3,2,5.000000,Ayunia Bridal dan Salon,0.683377,3.416883,[Wardah Lightening Liquid Foundation 25ml - ne...,6.22,0.279100,0.038657
4,3,3.000000,Rias Pengantin Criepno Lestari,1.106089,3.318267,[la tulipe cover foundation - suntan (sawo mat...,28.71,0.414050,0.013936
5,3,5.000000,Khaliza Make Up,0.650110,3.250549,[la tulipe cover foundation - natural (putih) ...,23.39,0.346331,0.014200
6,3,5.000000,Yulia Salon & Bridal,0.649355,3.246773,[la tulipe cover foundation - natural (putih) ...,25.48,0.271957,0.010270
7,2,4.000000,Laura Make Up & Bridal,0.685711,2.742846,[la tulipe cover foundation - suntan (sawo mat...,5.34,0.384264,0.060609
8,3,4.000000,Novi Make Up,0.649484,2.597935,[LT Pro smooth corrector cream foundation - na...,28.29,0.346331,0.011824


In [43]:
h_pivot.to_excel('highest.xlsx')

In [96]:
#lowest MAE
mae_eval[mae_eval.index(max(mae_eval))]

1.2585123717318591

In [97]:
#lowest
idx_low = mae_eval.index(max(mae_eval))
data_for_model.iloc[[selected_user[idx_low]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
101,61,maulina vira,Nonarika Make Up & Hairdo,-8.169978,113.728703,la tulipe cover foundation,suntan (sawo matang),medium,warm,0,-8.26545,113.530825,2,la tulipe cover foundation suntan sawo matang ...


In [98]:
h_pivot = testing_user(data_for_model, data_for_user, idx_low)
h_pivot[:9]

                              nama      mean  weight_user  weight_content
0                     maulina vira  1.087367     1.000000        1.015218
1           zainata hanum masruroh  0.803681     0.493333        1.018039
2                    anissa serina  0.776482     0.459055        1.012756
3         vika aprilia sinta putri  0.612461     0.266667        1.022120
4                 anggia pradita a  0.560683     0.216236        1.018631
5                        adera zia  0.511049     0.170561        1.018341
6                        dwi rahma  0.510795     0.170561        1.016308
7             donna rachel elkarim  0.293349     0.000000        1.047350
8                       yuna ayana  0.292658     0.000000        1.029976
9                  indah febrianti  0.292527     0.000000        1.026732
10             siti aisyah humaira  0.292518     0.000000        1.026526
11                   alevia frista  0.292416     0.000000        1.024013
12              nindy marsa wemona  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim,sim_final
0,3,4.666667,Ulfa Beauty MUA,0.982678,4.585830,[la tulipe cover foundation - natural (putih) ...,25.68,0.402294,0.015078
1,3,5.000000,Winda Wijaya MUA & Beauty Studio,0.899064,4.495322,[la tulipe cover foundation - suntan (sawo mat...,22.21,0.512470,0.022080
2,2,5.000000,Wedding Gallery Make Over,0.804147,4.020736,[la tulipe cover foundation - natural (putih) ...,18.79,0.452443,0.022862
3,1,5.000000,Laura Make Up & Bridal,0.779063,3.895317,[la tulipe cover foundation - natural (putih) ...,16.25,0.434197,0.025171
4,1,4.000000,Liyan Make Up Jember,0.771779,3.087117,[la tulipe cover foundation - natural (putih) ...,1047.88,0.434197,0.000414
5,2,5.000000,Sanggar Rias Cantik Tina,0.613674,3.068372,[la tulipe cover foundation - rachel 1 (kuning...,16.07,0.551607,0.032314
6,6,4.333333,Griya Putri Make Up,0.702081,3.042351,[la tulipe cover foundation - suntan (sawo mat...,21.03,0.482801,0.021916
7,2,5.000000,Aulia Rias Pengantin,0.560844,2.804220,[la tulipe cover foundation - suntan (sawo mat...,23.28,0.533361,0.021967
8,7,3.857143,Bonita MUA Jember,0.676166,2.608067,[LT Pro smooth corrector cream foundation - na...,22.04,0.379094,0.016454


In [33]:
h_pivot.to_excel('lowest.xlsx')

## COLLABORATIVE FILTERING

In [252]:
def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
    actual_colab = []
    table_colab = []
    rating_colab = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        
        nmf_model = NMF(2)
        f = nmf_model.fit_transform([feature])
        m = nmf_model.fit_transform(val_model)
        cos_sim_user = cosine_similarity(f, m)
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA', 'rating']]
        tmp_cos = np.zeros(len(part_of_model), dtype=float)
        for i in range(len(final_res)):
            for j in range(len(nama_user)):
                if final_res['nama'][i] == nama_user[j]:
                    tmp_cos[i] = cos_sim_user[0][j]
                    break

        final_res['mean_'] = tmp_cos
        higher_value = final_res.copy()
        higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        list_nama = higher_value.nama[:10].tolist()
        pivot = final_res.copy()
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_['weight']
        pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["weight"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_col.append(mae)

        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_colab.append(final_label)
        table_colab.append(pivot_)
        rating_colab.append(value_rating)
        
    # mean_mae_col = np.mean(mae_col)
    return table_colab, rating_colab, actual_colab

In [232]:
def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
    actual_colab = []
    table_colab = []
    rating_colab = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        knn = NearestNeighbors(metric='cosine', algorithm='brute')
        knn.fit(val_model)
        distances, indices = knn.kneighbors([feature], n_neighbors=10)
        list_nama = []
        for ln in indices[0]:
             list_nama.append(nama_user[ln])
        pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
        tmp_cos = np.empty(len(pivot), dtype=float)
        for i in range(len(pivot.nama)):
            for j in range(len(list_nama)):
                if pivot.nama[i] == list_nama[j]:
                    tmp_cos[i] = distances[0][j]
        pivot['cos'] = tmp_cos
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
        pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_col.append(mae)

        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_colab.append(final_label)
        table_colab.append(pivot_)
        rating_colab.append(value_rating)
        
    # mean_mae_col = np.mean(mae_col)
    return table_colab, rating_colab, actual_colab

In [253]:
colab_based_fil, value_rating_colab, actual_colab = colab_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [254]:
# colab_based_fil[colab_based_fil.score > 0.0]

In [255]:
colab_based_fil[0]

,count,mean,weight,predicted
nama_MUA,,,,
Laura Make Up & Bridal,1,5.000000,9.913960e-01,4.956980e+00
Aulia Rias Pengantin,2,5.000000,9.471476e-01,4.735738e+00
Liyan Make Up Jember,1,4.000000,9.913960e-01,3.965584e+00
Ulfa Beauty MUA,1,4.000000,9.913960e-01,3.965584e+00
Bonita MUA Jember,2,4.000000,9.887102e-01,3.954841e+00
Griya Putri Make Up,3,4.000000,9.618971e-01,3.847588e+00
Intan Rias Pengantin,2,4.000000,9.471476e-01,3.788591e+00
Larasnada Make Up,3,3.666667,9.896054e-01,3.628553e+00
Yulia Salon & Bridal,5,4.600000,4.573973e-01,2.104027e+00


## CONTENT BASED FILTERING

In [204]:
def content_evaluation(data_for_model, data_for_user, selected_user, query_use):
    actual_content = []
    table_content = []
    rating_content = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        get_user = part_of_model.groupby("nama").sim_num.agg(["mean"])
        get_user = get_user.sort_values(by=["mean"], ascending=False)
        list_nama = get_user.index[:10].tolist()
        pivot = part_of_model[part_of_model.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pivot.groupby("nama_MUA").total_mua.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
        pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_con.append(mae)

        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_content.append(final_label)
        table_content.append(pivot_)
        rating_content.append(value_rating)
        
    # mean_mae_con = np.mean(mae_con)
    return table_content, rating_content, actual_content

In [205]:
content_based_fil, value_rating_content, actual_content = content_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [206]:
# content_based_fil[content_based_fil.score > 0.0]

In [207]:
content_based_fil[0]

,count,mean,cos,predicted
nama_MUA,,,,
Gasebo Make Up Dekorasi,2,5.0,1.026626,5.133128
Rias Pengantin Sapphire,2,5.0,1.021740,5.108699
Khaliza Make Up,3,5.0,1.000652,5.003261
Rias Manten Mariyam,2,4.0,1.026941,4.107766
Bonita MUA Jember,2,4.0,1.016336,4.065342
Larasnada Make Up,2,4.0,1.011851,4.047405
Rosa Indah Griya Rias Pengantin,2,4.0,0.996796,3.987183
Novi Make Up,3,4.0,0.996672,3.986689
Salon Fatah,3,4.0,0.993339,3.973356


In [208]:
# merge_actual = [[i, j, z] for i,j,z in zip(actual_colab, actual_content, actual_hyb)]
# final_actual = []
# for i in range(len(merge_actual)):
#     tmp = []
#     for j in range(len(merge_actual[i][0])):
#         tmp.append(statistics.fmean([merge_actual[i][0][j], merge_actual[i][1][j], merge_actual[i][2][j]]))
#     final_actual.append(tmp)
    

In [256]:
def mae_evaluation(predicted_data, actual_data):
    maeUser = []
    rmseUser = []

    for i in range(len(predicted_data)):
        mae = mean_absolute_error(predicted_data[i], actual_data[i])
        rmse = mean_squared_error(predicted_data[i], actual_data[i], squared=False)
        maeUser.append(mae)
        rmseUser.append(rmse)
    mae = np.mean(maeUser)
    rmse = np.mean(rmseUser)

    return mae, rmse

In [257]:
mae_col, rmse_col = mae_evaluation(value_rating_colab, actual_colab)
mae_con, rmse_con = mae_evaluation(value_rating_content, actual_content)
mae_hyb, rmse_hyb = mae_evaluation(value_rating_hyb, actual_hyb)

In [258]:
data_evaluasi = {
    'Metode' : ['Collaborative Filtering', 'Content Based Filtering', 'Hybrid Filtering'],
    'MAE' : [mae_col, mae_con, mae_hyb],
    'RMSE' : [rmse_col, rmse_con, rmse_hyb]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE,RMSE
0,Hybrid Filtering,0.229251,0.810120
1,Content Based Filtering,0.256823,0.968129
2,Collaborative Filtering,0.838295,1.620447


In [218]:
# rating_test = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = final_actual[i][j]
#     rating_test.append(rating_dict)

# dataTesting = pd.DataFrame.from_dict(rating_test)
# dataTesting['user'] = query_user
# dataTesting.set_index(['user'])
# dataTesting.to_excel('dataTesting2.xlsx')

In [43]:
# predicted_rating = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = value_rating_hyb[i][j]
#     predicted_rating.append(rating_dict)

# predictedRating = pd.DataFrame.from_dict(predicted_rating)
# predictedRating['user'] = query_user
# predictedRating.set_index(['user'])
# predictedRating.to_excel('predictedRating2.xlsx')